# Лаба 5
Работаем

Качаем сет

In [78]:
import kagglehub

path = kagglehub.dataset_download("sanikamal/horses-or-humans-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Admin\.cache\kagglehub\datasets\sanikamal\horses-or-humans-dataset\versions\1


Задаём фиксированный seed, размер изображений, batch size и путь `PATH` до папки `horse-or-human`. Эти параметры будут использоваться во всех следующих шагах (генераторы, tf.data, модель).

In [ ]:
SEED = 501
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32
PATH = f"{path}/horse-or-human/"

- Достаём названия классов из `train/` и сортируем, чтобы порядок меток был стабильным.
- `ImageDataGenerator` делает нормализацию `rescale=1./255` и аугментации (повороты, сдвиги, зум, shear, флип), чтобы снизить переобучение.
- `flow_from_directory` создаёт батчи и формирует `class_indices` (какой класс -> 0/1).

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

classes = sorted([
    d for d in os.listdir(f"{PATH}/train")
    if os.path.isdir(os.path.join(PATH, "train", d))
])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.10,
    shear_range=0.05,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    f"{PATH}/train",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=True,
    seed=42,
    classes=classes
)

print("classes order:", classes)
print("train class_indices:", train_generator.class_indices)

Found 1027 images belonging to 2 classes.
classes order: ['horses', 'humans']
train class_indices: {'horses': 0, 'humans': 1}


Собираем пути к файлам и метки классов из `validation/`, затем делим их на две равные части:
- `val` — для мониторинга обучения и подбора порога
- `test` — для финальной оценки  
`stratify` сохраняет баланс классов в обеих выборках.

In [81]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

def list_files_and_labels(root_dir, classes):
    paths = []
    labels = []
    for i, cls in enumerate(classes):
        cls_dir = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_dir):
            raise FileNotFoundError(f"В {root_dir} нет папки класса: {cls_dir}")

        # подхватываем популярные расширения
        exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.webp")
        cls_files = []
        for e in exts:
            cls_files.extend(glob.glob(os.path.join(cls_dir, e)))

        paths.extend(cls_files)
        labels.extend([i] * len(cls_files))

    paths = np.array(paths)
    labels = np.array(labels, dtype=np.int32)
    return paths, labels

val_paths_all, val_labels_all = list_files_and_labels(f"{PATH}/validation", classes)

val_paths, test_paths, val_labels, test_labels = train_test_split(
    val_paths_all,
    val_labels_all,
    test_size=0.5,          # половина в test
    random_state=42,
    stratify=val_labels_all # чтобы классы делились ровно
)

print("val size:", len(val_paths), " test size:", len(test_paths))
print("val class balance:", np.bincount(val_labels))
print("test class balance:", np.bincount(test_labels))


val size: 128  test size: 128
val class balance: [64 64]
test class balance: [64 64]


In [82]:
def make_ds(paths, labels, batch_size, img_size=(IMG_HEIGHT, IMG_WIDTH), shuffle=False):
    paths = tf.constant(paths)
    labels = tf.cast(labels, tf.float32)

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths), seed=42, reshuffle_each_iteration=False)

    def _load(path, y):
        img_bytes = tf.io.read_file(path)
        img = tf.io.decode_image(img_bytes, channels=3, expand_animations=False)
        img.set_shape([None, None, 3])
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32) / 255.0
        return img, y

    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

val_ds = make_ds(val_paths, val_labels, BATCH_SIZE, shuffle=False)
test_ds = make_ds(test_paths, test_labels, BATCH_SIZE, shuffle=False)

Строим простую сверточную сеть:
несколько блоков `Conv2D + MaxPool`, затем `GlobalAveragePooling2D`, `Dropout` для регуляризации и `Dense(1, sigmoid)` для бинарной классификации. `model.summary()` — контроль архитектуры и числа параметров.

In [83]:
from tensorflow.keras import layers, models

def build_scratch_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)):
    inp = layers.Input(shape=input_shape)

    x = layers.Conv2D(16, 3, padding="same", activation="relu")(inp)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.3)(x)
    out = layers.Dense(1, activation="sigmoid")(x)

    return models.Model(inp, out)

model = build_scratch_cnn()
model.summary()


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 150, 150, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 75, 75, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 75, 75, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 37, 37, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 37, 37, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 18, 18, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 18, 18, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,569 (381.13 KB)

 Trainable params: 97,569 (381.13 KB)

 Non-trainable params: 0 (0.00 B)

In [84]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="roc_auc", curve="ROC")
    ]
)

In [85]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor="val_roc_auc", mode="max", patience=6, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_roc_auc", mode="max", factor=0.2, patience=3, min_lr=1e-6, verbose=1),
]

history = model.fit(
    train_generator,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 184ms/step - accuracy: 0.5024 - loss: 0.6899 - roc_auc: 0.5591 - val_accuracy: 0.5000 - val_loss: 0.7037 - val_roc_auc: 0.8179 - learning_rate: 0.0010
Epoch 2/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 181ms/step - accuracy: 0.6018 - loss: 0.6678 - roc_auc: 0.6369 - val_accuracy: 0.5000 - val_loss: 1.2937 - val_roc_auc: 0.8549 - learning_rate: 0.0010
Epoch 3/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 181ms/step - accuracy: 0.6592 - loss: 0.6288 - roc_auc: 0.7061 - val_accuracy: 0.5312 - val_loss: 0.6518 - val_roc_auc: 0.6761 - learning_rate: 0.0010
Epoch 4/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 185ms/step - accuracy: 0.7196 - loss: 0.5378 - roc_auc: 0.8020 - val_accuracy: 0.5312 - val_loss: 0.7325 - val_roc_auc: 0.8354 - learning_rate: 0.0010
Epoch 5/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.7310 - loss: 0.5278 - roc_auc: 0.8235
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 181ms/step - accuracy

- Считаем вероятности на `val`, ROC-AUC и строим ROC-кривую.
- Подбираем лучший порог по критерию Юдена: максимизируем `tpr - fpr`.
- На `val` считаем accuracy/confusion matrix/report при этом пороге.
- На `test` считаем ROC-AUC и метрики при фиксированном пороге с `val` (честно: тест не участвует в подборе порога).

In [ ]:
import numpy as np
from sklearn.metrics import (
    roc_curve, roc_auc_score,
    accuracy_score, confusion_matrix, classification_report
)

p_val = model.predict(val_ds, verbose=0).ravel()
y_val = val_labels

val_auc = roc_auc_score(y_val, p_val)
print("VAL ROC-AUC:", val_auc)

fpr, tpr, thr = roc_curve(y_val, p_val)
j = tpr - fpr

best_idx = int(np.argmax(j))
best_thr = float(thr[best_idx])

if not np.isfinite(best_thr):
    finite = np.isfinite(thr)
    best_idx = int(np.argmax(j[finite]))
    best_thr = float(thr[finite][best_idx])

y_val_pred = (p_val >= best_thr).astype(int)

print("\nBest threshold (from VAL):", best_thr)
print("VAL Accuracy @ best_thr:", accuracy_score(y_val, y_val_pred))
print("VAL Confusion matrix:\n", confusion_matrix(y_val, y_val_pred))
print("VAL Report:\n", classification_report(y_val, y_val_pred, digits=4))

p_test = model.predict(test_ds, verbose=0).ravel()
y_test = test_labels

test_auc = roc_auc_score(y_test, p_test)  # AUC порог не нужен
y_test_pred = (p_test >= best_thr).astype(int)

print("\nTEST ROC-AUC:", test_auc)
print("TEST Accuracy @ fixed best_thr:", accuracy_score(y_test, y_test_pred))
print("TEST Confusion matrix:\n", confusion_matrix(y_test, y_test_pred))
print("TEST Report:\n", classification_report(y_test, y_test_pred, digits=4))


VAL ROC-AUC: 0.87158203125

Best threshold (from VAL): 0.927260160446167
VAL Accuracy @ best_thr: 0.8203125
VAL Confusion matrix:
 [[59  5]
 [18 46]]
VAL Report:
               precision    recall  f1-score   support

           0     0.7662    0.9219    0.8369        64
           1     0.9020    0.7188    0.8000        64

    accuracy                         0.8203       128
   macro avg     0.8341    0.8203    0.8184       128
weighted avg     0.8341    0.8203    0.8184       128


TEST ROC-AUC: 0.888916015625
TEST Accuracy @ fixed best_thr: 0.859375
TEST Confusion matrix:
 [[59  5]
 [13 51]]
TEST Report:
               precision    recall  f1-score   support

           0     0.8194    0.9219    0.8676        64
           1     0.9107    0.7969    0.8500        64

    accuracy                         0.8594       128
   macro avg     0.8651    0.8594    0.8588       128
weighted avg     0.8651    0.8594    0.8588       128



### Вывод по результатам
Модель показывает хорошее качество ранжирования. Порого подбирается корректно, причем на тестовых данных результат оказался лучше, чем на валидационных(хотя это просто случайность, что в тестовых данных меньше выбросов).